# Preprocessing

## Imports

In [12]:
import pandas as pd
import os

## Load datasets

In [13]:
sales_df = pd.read_csv('./datasets/SalesForCourse_quizz_table.csv')
spending_df = pd.read_csv('./datasets/spending_patterns_detailed.csv')

## Spending patterns

### Filter the columns

In [14]:
spending_df = spending_df[['Customer ID', 'Category', 'Transaction Date', 'Total Spent']]
spending_df.rename(columns={
    'Customer ID': 'customer',
    'Category': 'category',
    'Transaction Date': 'date',
    'Total Spent': 'amount'
}, inplace=True)

### Create categories

In [15]:
# Función para seleccionar un único usuario por categoría
def select_user_by_category(df):
    """
    Selecciona un único usuario representativo para cada categoría.
    """
    user_categories = {}

    # Tacaño: Usuario con el gasto total más bajo
    thrifty_user = df.groupby('customer').sum().sort_values('amount').head(1).index[0]
    user_categories['miserly'] = df[df['customer'] == thrifty_user]

    # Estudiante: Usuario con mayor proporción de gastos en 'Groceries' y 'Food'
    student_user = (
        df[df['category'].isin(['Groceries', 'Food'])]
        .groupby('customer').sum()
        .sort_values('amount', ascending=False)
        .head(1).index[0]
    )
    user_categories['student'] = df[df['customer'] == student_user]

    # Derrochador: Usuario con el gasto total más alto
    big_spender = df.groupby('customer').sum().sort_values('amount', ascending=False).head(1).index[0]
    user_categories['wasteful'] = df[df['customer'] == big_spender]

    # Familiar: Usuario con mayor proporción de gastos en 'Housing and Utilities' y 'Groceries'
    family_user = (
        df[df['category'].isin(['Housing and Utilities', 'Groceries'])]
        .groupby('customer').sum()
        .sort_values('amount', ascending=False)
        .head(1).index[0]
    )
    user_categories['familiar'] = df[df['customer'] == family_user]

    # Aventurero: Usuario con mayor proporción de gastos en 'Travel' y 'Hobbies'
    adventurous_user = (
        df[df['category'].isin(['Travel', 'Hobbies'])]
        .groupby('customer').sum()
        .sort_values('amount', ascending=False)
        .head(1).index[0]
    )
    user_categories['adventurous'] = df[df['customer'] == adventurous_user]

    return user_categories

### Saving the final datasets

In [16]:
user_categories = select_user_by_category(spending_df)

output_dir = './datasets/users'
os.makedirs(output_dir, exist_ok=True)

for category, data in user_categories.items():
    data.to_csv(f'{output_dir}/{category.lower()}.csv', index=False)

## Sales For Course

### Filter the columns

In [9]:
sales_df = sales_df[['Date', 'Product Category', 'Quantity', 'Unit Cost', 'Cost']]
sales_df.rename(columns={
    'Date': 'date',
    'Product Category': 'category',
    'Quantity': 'quantity',
    'Unit Cost': 'unit_cost',
    'Cost': 'total_cost'
}, inplace=True)

### Create categories

In [10]:
# Función para seleccionar un único usuario por categoría
def select_user_by_sales_category(df):
    """
    Selecciona un único usuario representativo para cada categoría basada en patrones de compra.
    """
    user_categories = {}

    # Ahorro en accesorios: Usuario con mayor gasto en 'Accessories'
    accessories_user = (
        df[df['category'] == 'Accessories']
        .groupby('category').sum()
        .sort_values('total_cost', ascending=False)
        .head(1).index[0]
    )
    user_categories['Ahorro en accesorios'] = df[df['category'] == accessories_user]

    # Comprador de ropa: Usuario con mayor gasto en 'Clothing'
    clothing_user = (
        df[df['category'] == 'Clothing']
        .groupby('category').sum()
        .sort_values('total_cost', ascending=False)
        .head(1).index[0]
    )
    user_categories['Comprador de ropa'] = df[df['category'] == clothing_user]

    # Comprador de alto volumen: Usuario que compra la mayor cantidad de productos
    high_volume_user = df.groupby('category').sum().sort_values('quantity', ascending=False).head(1).index[0]
    user_categories['Comprador de alto volumen'] = df[df['category'] == high_volume_user]

    # Comprador de bajo costo: Usuario que compra productos con el menor costo unitario promedio
    low_cost_user = df.groupby('category').mean().sort_values('unit_cost').head(1).index[0]
    user_categories['Comprador de bajo costo'] = df[df['category'] == low_cost_user]

    return user_categories

### Saving the final datasets

In [11]:
# Seleccionar usuarios representativos
sales_user_categories = select_user_by_sales_category(sales_df)

output_dir = './datasets/users'
os.makedirs(output_dir, exist_ok=True)

for category, data in sales_user_categories.items():
    data.to_csv(f'{output_dir}/{category.lower().replace(" ", "_")}.csv', index=False)

TypeError: agg function failed [how->mean,dtype->object]